In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
df = pd.read_csv('data_halodoc_unordered.csv', delimiter=';')
df.head()

,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...


In [6]:
df.columns

Index(['name', 'price', 'Aturan Pakai', 'Dosis', 'Efek Samping',
       'Golongan Produk', 'Indikasi Umum', 'Kemasan', 'Komposisi',
       'Kontra Indikasi', 'Manufaktur', 'No. Registrasi', 'Perhatian', 'uses'],
      dtype='object')

In [9]:
kolom_teks = [
    'name', 'Aturan Pakai', 'Dosis', 'Efek Samping',
    'Golongan Produk', 'Indikasi Umum', 'Kemasan', 'Komposisi',
    'Kontra Indikasi', 'Manufaktur', 'No. Registrasi', 'Perhatian', 'uses']

df['kombinasi_teks'] = df[kolom_teks].fillna('').agg(' '.join, axis=1)
df['kombinasi_teks'].head()

0    Sildenafil Citrate 50 mg 4 Tablet Dikonsumsi s...
1    Viagra 50 mg 1 Tablet Dikonsumsi satu jam sebe...
2    Ericfil Tablet 100 mg Dikonsumsi satu jam sebe...
3    Cialis Tablet 10 mg Diberikan sebelum atau set...
4    Ericfil 50 mg Odf Cara penggunaan Ericfil ODF:...
Name: kombinasi_teks, dtype: object

## Preprocessing

In [18]:
def preprocessing(teks):
    if pd.isna(teks):
        return ''
    teks = str(teks).lower()
    teks = re.sub(r'[^a-z0-9\s]', '', teks)
    tokens = word_tokenize(teks)
    stop_words = set(stopwords.words('indonesian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['prepro_teks'] = df['kombinasi_teks'].apply(preprocessing)
df

,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses,kombinasi_teks,prepro_teks
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...,Sildenafil Citrate 50 mg 4 Tablet Dikonsumsi s...,sildenafil citrat tablet dikonsumsi jam aktivi...
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...,Viagra 50 mg 1 Tablet Dikonsumsi satu jam sebe...,viagra tablet dikonsumsi jam aktivita seksual ...
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...,Ericfil Tablet 100 mg Dikonsumsi satu jam sebe...,ericfil tablet 100 dikonsumsi jam aktivita sek...
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...,Cialis Tablet 10 mg Diberikan sebelum atau set...,ciali tablet makan penggunaan obat sesuai petu...
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...,Ericfil 50 mg Odf Cara penggunaan Ericfil ODF:...,ericfil odf penggunaan ericfil odf buka kemasa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,Rp2.200.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan otot, mempertahankan mass...",Pack @ 4.54 Kg,"Protein Blend (Whey Protein Isolate, Whey Prot...",NaN,Vita Shopindo,NaN,NaN,Optimum Nutrition On Whey Gold Standard Vanill...,Optimum Nutrition Whey Gold Standard Vanilla 1...,optimum nutrit whey gold standard vanilla diko...
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,Rp1.150.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan o

## Query

In [31]:
corpus = df['prepro_teks'].tolist()
nama_obat = df['name'].tolist()

print(corpus[:5])
print(nama_obat[:5])

['sildenafil citrat tablet dikonsumsi jam aktivita seksual disesuaikan kebutuhan penggunaan obat sesuai petunjuk dokter dewasa sesuai kebutuhan jam aktivita seksual disesuaikan kebutuhan maksim 100 mghari sakit kepala pana kemerahan kulit wajah dispepsia hidung tersumbat gangguan penglihatan diar puse ruam kulit isk obat kera merah informasi obat kalangan medi terapi disfungsi ereksi pria dewasa du strip tablet sildenafil citrat hipersensitivita pasien nitrat organik regul intermiten gener manufactur bpom gkl1433531717a1 obat obat generik nomor registrasi berbeda sesuai ketersediaan stock apotek resep dokter pasien pria dianjurkan aktivita seksual penyakit pasien infark miokard stroke aritmia mengancam nyawa hipertensi td170110 hipotensi td9050 istirahat riwayat gangguan sildenafil citrat obat mengobati gangguan fungsi seksual pria disfungsi ereksi obat meningkatkan aliran darah alat kelamin pria mempertahankan ereksi berhubungan seksual', 'viagra tablet dikonsumsi jam aktivita seksual

In [33]:
query_pendek = ['obat jantung', 'obat stres', 'krim wajah']
query_panjang = [
    'obat oles untuk luka bakar ringan yang tidak perih',
    'obat herbal apa yang sudah terdaftar BPOM',
    'obat flu untuk balita']

query_pendek_prepro = [preprocessing(q) for q in query_pendek]
query_panjang_prepro = [preprocessing(q) for q in query_panjang]

## Information Retrieval

#### Boolean Retrieval